<a href="https://colab.research.google.com/github/GeraudBourdin/llm-scripts/blob/main/convert_context_qa_load_v2_To_mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### convert context_qa.load_v2 To mistral

In [1]:
!pip install transformers mistral_common datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.9/704.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall:

In [ ]:
from datasets import Dataset, load_dataset
import json
from typing import Dict
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

mistralTokenizer = MistralTokenizer.v3()

## variable servant a connaître le max de tokens de notre dataset
max_dataset_token_length = 0

def format_data(data: Dict) -> Dict:
    global max_dataset_token_length

    dataset = {}
    prompt = ""
    answer = ""

    if "system_prompt" in data:
        prompt += data["system_prompt"]
        prompt += "\n"

    if "context" in data:
        prompt += data["context"]
        prompt += "\n"

    if "question" in data:
        prompt += data["question"]

    if "answer" in data:
        answer += data["answer"]

    if "response" in data:
        answer += data["response"]

    chat = [
      {"role": "user", "content": prompt},
      {"role": "assistant", "content": answer},
      {"role": "user", "content": ""},
    ]
    tokenized = mistralTokenizer.encode_chat_completion(ChatCompletionRequest(messages=chat))
    dataset['chat'] = chat
    dataset['json'] = data
    dataset['text'] = tokenized.text.replace('[INST][/INST]', '')
    dataset['length'] = len(tokenized.tokens)
    ## si jamais la taille de notre instruction dépasse la derniere
    if dataset['length'] > max_dataset_token_length :
      max_dataset_token_length = dataset['length']
    return dataset



trainDatas = []
## on ouvre le fichier
with open('train.jsonl', 'r') as f:
  ## on boucle sur chacune des lignes
  for line in f:
    ## on mets dans data un equivalent de json_decode()
    data = json.loads(line)
    ## on passe notre array json a note fonction chargée d'encoder nos informations
    formatted_data = format_data(data)
    ## on pousse dans notre array les informations décodés
    trainDatas.append(formatted_data)

## On cree un objet Dataset (de hugging face) contenant notre array trainDatas
train_dataset = Dataset.from_dict({'text': [item['text'] for item in trainDatas] })


print(train_dataset[2])
print(max_dataset_token_length)